Zomato and Gourmet Traveller Data Plotted using Google Maps

In [14]:
#Dependencies
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import gmaps
from api_key import apikey 
from api_key import g_key 
from pytrends.request import TrendReq
%matplotlib widget

In [15]:
#Set Search URL
search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id=259&entity_type=city"

#Create Requests
print(requests.get(search_url, headers={"user-key" : apikey}))

#Get Data and Convert into JSON
data = requests.get(search_url, headers={"user-key" : apikey}).json()

<Response [200]>


In [16]:
#Set City IDs
cityid = [296,259,260]

#Create Empty Lists
rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_lat_list = []
rest_long_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []

resto_num = 20
loop_num = 0
show_page = 0

#Create For Loop
for citynum in cityid:
    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
    data = requests.get(search_url, headers={"user-key" : apikey}).json()
    for page in range(5):
        for count in range(resto_num):
            try: 
                rest_id = data["restaurants"][count]["restaurant"]["id"]
                rest_name = data["restaurants"][count]["restaurant"]["name"]
                rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
                rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
                rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
                rest_lat = data["restaurants"][count]["restaurant"]["location"]["latitude"]
                rest_long = data["restaurants"][count]["restaurant"]["location"]["longitude"]
                rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
                rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
                rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
                
                #Check if they Deliver 
                theydelivery = False
                for doesdelivery in rest_delivery:
                    if doesdelivery == 'Delivery':
                        theydelivery = True

                if theydelivery == True:
                    rest_delivery = "Yes"
                else:
                    rest_delivery = "No"

                #Append Results to List
                rest_id_list.append(rest_id)
                rest_name_list.append(rest_name)
                rest_cuis_list.append(rest_cuis)
                rest_local_list.append(rest_local)
                rest_city_list.append(rest_city)
                rest_lat_list.append(rest_lat)
                rest_long_list.append(rest_long)
                rest_rate_list.append(rest_rate)
                rest_ratenum_list.append(rest_rate_num)
                rest_delivery_list.append(rest_delivery)
                
                #Repeat
                if count == 19:
                    show_page = show_page + 20
                    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                    data = requests.get(search_url, headers={"user-key" : apikey}).json()
                    count = 0
            except:
                print("error")
    count = 0
    page = 0
    show_page = 0


#Check for Errors
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Latitude":rest_lat_list,
        "Longitude":rest_long_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_lat_list))
    print(len(rest_long_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
    
#Drop Duplicate Results    
restaurants_df = restaurants_df.drop_duplicates()

In [17]:
#Sort DataFrame by Rating
restaurants_df = restaurants_df.sort_values(by=["Rating"],ascending=False)

#Print DataFrame
reshaped = \
(restaurants_df.set_index(restaurants_df.columns.drop('Cuisine',1).tolist())
   .Cuisine.str.split(', ', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Cuisine'})
   .loc[:, restaurants_df.columns]
)

reshaped_df = pd.DataFrame(reshaped)

#Get Top Restaurants in Perth
perth_top = reshaped_df.loc[reshaped_df["City"]=="Perth"]

#Get Top Restaurants in Melbourne
melbourne_top = reshaped_df.loc[reshaped_df["City"]=="Melbourne"]

#Get Top Restaurants in Sydney
sydney_top = reshaped_df.loc[reshaped_df["City"]=="Sydney"]

In [18]:
#Set Longitude and Latitude
locations = melbourne_top[["Latitude", "Longitude"]].astype(float)

#Set 'Rating' as Weights
rating = melbourne_top["Rating"].astype(float)

#Plot Map
figure = gmaps.figure()

#Set Heat Map Layer Parameters
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)
#Set Heat Map Layer
figure.add_layer(heat_layer)

#Display Map
figure

Figure(layout=FigureLayout(height='420px'))

In [19]:
#Set Longitude and Latitude
locations = sydney_top[["Latitude", "Longitude"]].astype(float)

#Set 'Rating' as Weights
rating = sydney_top["Rating"].astype(float)

#Plot Map
figure = gmaps.figure()

#Set Heat Map Layer Parameters
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)
#Set Heat Map Layer 
figure.add_layer(heat_layer)

#Display Map
figure

Figure(layout=FigureLayout(height='420px'))

In [20]:
#Set Longitude and Latitude
locations = perth_top[["Latitude", "Longitude"]].astype(float)

#Set 'Rating' as Weights
rating = perth_top["Rating"].astype(float)

#Plot Map
figure = gmaps.figure()

#Set Heat Map Layer Parameters
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)
#Set Heat Map Layer 
figure.add_layer(heat_layer)

#Display Map
figure

Figure(layout=FigureLayout(height='420px'))

In [21]:
#Get Zomato Restaurant CSV file
multi_city = "Multi_Restaurant_Data.csv"

#Read Zomato Restaurant CSV file
multi_df = pd.read_csv(multi_city)

#Get Gourmet Traveller 2019 Ratings CSV file
top_resto_rating = "GT_Top_Restaurants.csv"

#Read Gourmet Traveller 2019 Ratings CSV file
top_resto_rating_df = pd.read_csv(top_resto_rating)

#Get Gourmet Traveller 2019 without Ratings CSV file
top_resto = "GT_Top_Resto.csv"

#Read Gourmet Traveller 2019 without Ratings CSV file
top_resto_df = pd.read_csv(top_resto)

#Merge Zomato Restaurant and Gourmet Traveller 2019 without Ratings DataFrames
merged_top_resto = pd.merge(multi_df, top_resto_df, on='Name', how='inner')

In [22]:
#Set Longitude and Latitude
locations = top_resto_rating_df[["Latitude", "Longitude"]].astype(float)

#Set 'Rating' as Weights
rating = top_resto_rating_df["Rating"].astype(float)

#Plot Map
figure = gmaps.figure()

#Set Heat Map Layer Parameters
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)
#Set Heat Map Layer 
figure.add_layer(heat_layer)

#Display Map
figure

Figure(layout=FigureLayout(height='420px'))

In [23]:
#Set Longitude and Latitude
locations = merged_top_resto[["Latitude", "Longitude"]].astype(float)

#Set 'Rating' as Weights
rating = merged_top_resto["Rating"].astype(float)

#Plot Map
figure = gmaps.figure()

#Set Heat Map Layer Parameters
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)

#Add Heat Map Layer
figure.add_layer(heat_layer)

#Add Restaurant Markers to Heat Map
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
#Store the DataFrame Row
Name_info = [info_box_template.format(**row) for index, row in merged_top_resto.iterrows()]
locations = merged_top_resto[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations, info_box_content=Name_info)

#Add Marker Layer on top of Heat Map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

#Display Map
figure

Figure(layout=FigureLayout(height='420px'))